## Acquire and Prep

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import env

**SQL query to use for the MVP model**

In [2]:
query = """
SELECT 
bathroomcnt AS bath_count,
bedroomcnt AS bedroom_count,
calculatedfinishedsquarefeet AS sqft,
fips,
taxvaluedollarcnt AS assessed_tax_value,                                                                            
(taxamount / taxvaluedollarcnt) AS tax_rate
FROM properties_2017
JOIN predictions_2017 AS pr USING(`parcelid`)
WHERE (transactiondate >= '2017-05-01' AND transactiondate <= '2017-06-30') AND
propertylandusetypeid = 261;"""

In [3]:
#add get_db_url to wrangle.py
URL = get_db_url("zillow")

In [4]:
zillow = pd.read_sql(query, URL)

**Take a peek at the data**

In [5]:
zillow.sample(4)

,bath_count,bedroom_count,sqft,fips,assessed_tax_value,tax_rate
1520,2.0,4.0,1472.0,6059.0,580000.0,0.010212
12252,2.5,4.0,2710.0,6059.0,162196.0,0.012643
13382,2.0,4.0,1567.0,6037.0,300963.0,0.012554
8903,2.0,3.0,1624.0,6037.0,193750.0,0.016466


**Bring in the fips**

In [6]:
FIPS = pd.read_csv("FIPS.txt", sep="\t")

In [7]:
FIPS

,FIPS,Name,State
0,1001,Autauga,AL
1,1003,Baldwin,AL
2,1005,Barbour,AL
3,1007,Bibb,AL
4,1009,Blount,AL
...,...,...,...
3227,72151,Yabucoa,PR
3228,72153,Yauco,PR
3229,78010,St. Croix,VI
3230,78020,St. John,VI


In [8]:
zillow = pd.merge(left=zillow, right=FIPS, left_on="fips", right_on="FIPS")

In [9]:
zillow

,bath_count,bedroom_count,sqft,fips,assessed_tax_value,tax_rate,FIPS,Name,State
0,2.0,3.0,1458.0,6037.0,136104.0,0.017045,6037,Los Angeles,CA
1,1.0,2.0,1421.0,6037.0,35606.0,0.015270,6037,Los Angeles,CA
2,2.0,3.0,1650.0,6037.0,614000.0,0.012497,6037,Los Angeles,CA
3,1.0,2.0,693.0,6037.0,274237.0,0.011915,6037,Los Angeles,CA
4,0.0,0.0,1378.0,6037.0,168828.0,0.012648,6037,Los Angeles,CA
...,...,...,...,...,...,...,...,...,...
15031,2.0,3.0,1575.0,6111.0,506000.0,0.010707,6111,Ventura,CA
15032,2.0,3.0,1645.0,6111.0,230986.0,0.012363,6111,Ventura,CA
15033,2.0,3.0,1308.0,6111.0,255407.0,0.010919,6111,Ventura,CA
15034,2.5,4.0,2314.0,6111.0,485364.0,0.019674,6111,Ventura,CA


In [10]:
zillow.rename(columns = {"Name": "county_name"}, inplace = True)

Check the datatypes and info

In [11]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15036 entries, 0 to 15035
Data columns (total 9 columns):
bath_count            15036 non-null float64
bedroom_count         15036 non-null float64
sqft                  15012 non-null float64
fips                  15036 non-null float64
assessed_tax_value    15036 non-null float64
tax_rate              15035 non-null float64
FIPS                  15036 non-null int64
county_name           15036 non-null object
State                 15036 non-null object
dtypes: float64(6), int64(1), object(2)
memory usage: 1.1+ MB


let's check to see how many zeros are in bed and bath

In [12]:
zillow.bath_count.value_counts()

2.0     6182
3.0     3114
1.0     2652
2.5     1155
4.0      671
3.5      302
1.5      240
5.0      233
4.5      217
6.0      102
5.5       65
0.0       40
7.0       25
6.5       15
8.0       14
7.5        4
9.0        3
11.0       1
10.0       1
Name: bath_count, dtype: int64

In [13]:
zillow.bedroom_count.value_counts()

3.0     6668
4.0     4423
2.0     2350
5.0     1158
6.0      183
1.0      177
0.0       42
7.0       27
12.0       3
9.0        2
11.0       1
10.0       1
8.0        1
Name: bedroom_count, dtype: int64

I'm going to drop the zeros in bed and bath to get my MVP out as fast as possible.

- If thime permits later I might try to replace with mean or mode

In [14]:
zillow.bath_count.replace(0, np.nan, inplace=True)

In [15]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15036 entries, 0 to 15035
Data columns (total 9 columns):
bath_count            14996 non-null float64
bedroom_count         15036 non-null float64
sqft                  15012 non-null float64
fips                  15036 non-null float64
assessed_tax_value    15036 non-null float64
tax_rate              15035 non-null float64
FIPS                  15036 non-null int64
county_name           15036 non-null object
State                 15036 non-null object
dtypes: float64(6), int64(1), object(2)
memory usage: 1.1+ MB


In [16]:
zillow.bedroom_count.replace(0, np.nan, inplace=True)

In [17]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15036 entries, 0 to 15035
Data columns (total 9 columns):
bath_count            14996 non-null float64
bedroom_count         14994 non-null float64
sqft                  15012 non-null float64
fips                  15036 non-null float64
assessed_tax_value    15036 non-null float64
tax_rate              15035 non-null float64
FIPS                  15036 non-null int64
county_name           15036 non-null object
State                 15036 non-null object
dtypes: float64(6), int64(1), object(2)
memory usage: 1.1+ MB


In [18]:
zillow.dropna(inplace=True)

In [19]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14989 entries, 0 to 15035
Data columns (total 9 columns):
bath_count            14989 non-null float64
bedroom_count         14989 non-null float64
sqft                  14989 non-null float64
fips                  14989 non-null float64
assessed_tax_value    14989 non-null float64
tax_rate              14989 non-null float64
FIPS                  14989 non-null int64
county_name           14989 non-null object
State                 14989 non-null object
dtypes: float64(6), int64(1), object(2)
memory usage: 1.1+ MB
